# Ejercicios resueltos del libro *Programming Massively Parallel Processors: A Hands-on Approach*, capítulo 3: *Scalable Parallel Execution*

## Ejercicio 1: Suma de matrices

In [25]:
%%writefile ej31.cu

#include <iostream>

__global__
void mat_add_B(float* C, float* A, float* B, int n){
  int i = blockIdx.y * blockDim.y + threadIdx.y;
  int j = blockIdx.x * blockDim.x + threadIdx.x;

  if (i >= n || j >= n) return;

  C[i * n + j] = A[i * n + j] + B[i * n + j];
}

__global__
void mat_add_C(float *C, float *A, float *B, int n)
{
  int i = blockIdx.x * blockDim.x + threadIdx.x;

  if (i >= n) return;

  for (int j = 0; j < n; ++j)
    C[i * n + j] = A[i * n + j] + B[i * n + j];
}

__global__
void mat_add_D(float *C, float *A, float *B, int n)
{
  int j = blockIdx.x * blockDim.x + threadIdx.x;

  if (j >= n) return;

  for (int i = 0; i < n; ++i)
    C[i * n + j] = A[i * n + j] + B[i * n + j];
}

int main(){
  int n = 128;

  float *host_A, *host_B, *host_C;
  float *dev_A, *dev_B, *dev_C;

  // allocating memory for the input and output matrices
  host_A = (float*)malloc(n * n * sizeof(float));
  host_B = (float*)malloc(n * n * sizeof(float));
  host_C = (float*)malloc(n * n * sizeof(float));

  for(int i = 0; i < n * n; i++){
    host_A[i] = rand() % 100;
    host_B[i] = rand() % 100;
  }

  cudaMalloc(&dev_A, n * n * sizeof(float));
  cudaMalloc(&dev_B, n * n * sizeof(float));
  cudaMalloc(&dev_C, n * n * sizeof(float));

  // transferring input data to device
  cudaMemcpy(dev_A, host_A, n * n * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(dev_B, host_B, n * n * sizeof(float), cudaMemcpyHostToDevice);

  dim3 block(32, 16);
  dim3 grid((n + block.x - 1)/block.x,
            (n + block.y - 1)/block.y);

  //--------launch the kernel--------
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  cudaEventRecord(start);

  mat_add_B<<<grid, block>>>(dev_C, dev_A, dev_B, n);
  //mat_add_C<<<grid, block>>>(dev_C, dev_A, dev_B, n);
  //mat_add_D<<<grid, block>>>(dev_C, dev_A, dev_B, n);

  cudaDeviceSynchronize();

  cudaEventRecord(stop);
  cudaEventSynchronize(stop);

  float ms = 0.0f;
  cudaEventElapsedTime(&ms, start, stop);

  std::cout << "Tiempo GPU: " << ms << " ms\n";

  cudaEventDestroy(start);
  cudaEventDestroy(stop);
  //---------------------------------

  // transferring the output data to host
  cudaMemcpy(host_C, dev_C, n * n * sizeof(float), cudaMemcpyDeviceToHost);

  // test
  for(int i = 0; i < n; ++i){
    for(int j = 0; j < n; ++j)
      std::cout << host_C[i * n + j] << " ";
    std::cout << std::endl;
  }

  // freeing the device memory for the input and output data
  cudaFree(dev_A);
  cudaFree(dev_B);
  cudaFree(dev_C);

  free(host_A);
  free(host_B);
  free(host_C);

  return 0;
}


Overwriting ej31.cu


In [26]:
%%shell

# mat_add_b

nvcc -arch=compute_75 -code=sm_75 ej31.cu -o ej31
./ej31

Tiempo GPU: 0.122976 ms
169 92 128 178 70 89 149 89 66 108 79 96 112 85 102 31 80 136 149 53 102 40 121 122 85 39 171 129 132 177 30 111 41 75 70 119 127 81 98 93 95 154 172 54 153 92 144 51 112 133 165 112 68 99 109 108 81 127 154 63 69 48 168 156 36 123 150 151 89 43 99 61 22 138 23 89 119 63 97 76 167 92 91 93 98 38 143 134 92 95 83 65 82 95 153 66 128 46 54 57 80 106 51 46 22 116 86 113 67 153 189 97 134 44 140 151 106 104 59 59 50 142 114 62 21 6 91 83 
56 70 87 24 140 61 86 36 151 129 139 104 111 127 123 180 195 132 81 185 92 124 174 118 43 101 97 99 120 68 46 129 7 85 76 118 25 50 107 33 77 142 44 91 177 114 164 122 110 121 82 111 188 114 90 55 87 98 148 149 105 102 163 98 88 17 115 85 46 8 105 110 117 80 156 44 101 36 81 144 113 97 46 185 139 36 160 145 111 148 163 86 96 35 64 55 98 58 191 142 41 31 81 85 110 69 131 42 33 113 114 78 77 109 138 155 187 87 119 122 110 58 103 77 84 90 115 163 
111 116 81 82 63 98 46 120 68 107 136 117 131 124 84 98 69 102 91 79 163 96 53 175 104 1

In [21]:
%%shell

# mat_add_c

nvcc -arch=compute_75 -code=sm_75 ej31.cu -o ej31
./ej31

Tiempo GPU: 0.809376 ms
169 92 128 178 70 89 149 89 66 108 79 96 112 85 102 31 80 136 149 53 102 40 121 122 85 39 171 129 132 177 30 111 41 75 70 119 127 81 98 93 95 154 172 54 153 92 144 51 112 133 165 112 68 99 109 108 81 127 154 63 69 48 168 156 36 123 150 151 89 43 99 61 22 138 23 89 119 63 97 76 167 92 91 93 98 38 143 134 92 95 83 65 82 95 153 66 128 46 54 57 80 106 51 46 22 116 86 113 67 153 189 97 134 44 140 151 106 104 59 59 50 142 114 62 21 6 91 83 
56 70 87 24 140 61 86 36 151 129 139 104 111 127 123 180 195 132 81 185 92 124 174 118 43 101 97 99 120 68 46 129 7 85 76 118 25 50 107 33 77 142 44 91 177 114 164 122 110 121 82 111 188 114 90 55 87 98 148 149 105 102 163 98 88 17 115 85 46 8 105 110 117 80 156 44 101 36 81 144 113 97 46 185 139 36 160 145 111 148 163 86 96 35 64 55 98 58 191 142 41 31 81 85 110 69 131 42 33 113 114 78 77 109 138 155 187 87 119 122 110 58 103 77 84 90 115 163 
111 116 81 82 63 98 46 120 68 107 136 117 131 124 84 98 69 102 91 79 163 96 53 175 104 1

In [18]:
%%shell

# mat_add_d

nvcc -arch=compute_75 -code=sm_75 ej31.cu -o ej31
./ej31

Tiempo GPU: 0.182304 ms
169 92 128 178 70 89 149 89 66 108 79 96 112 85 102 31 80 136 149 53 102 40 121 122 85 39 171 129 132 177 30 111 41 75 70 119 127 81 98 93 95 154 172 54 153 92 144 51 112 133 165 112 68 99 109 108 81 127 154 63 69 48 168 156 36 123 150 151 89 43 99 61 22 138 23 89 119 63 97 76 167 92 91 93 98 38 143 134 92 95 83 65 82 95 153 66 128 46 54 57 80 106 51 46 22 116 86 113 67 153 189 97 134 44 140 151 106 104 59 59 50 142 114 62 21 6 91 83 
56 70 87 24 140 61 86 36 151 129 139 104 111 127 123 180 195 132 81 185 92 124 174 118 43 101 97 99 120 68 46 129 7 85 76 118 25 50 107 33 77 142 44 91 177 114 164 122 110 121 82 111 188 114 90 55 87 98 148 149 105 102 163 98 88 17 115 85 46 8 105 110 117 80 156 44 101 36 81 144 113 97 46 185 139 36 160 145 111 148 163 86 96 35 64 55 98 58 191 142 41 31 81 85 110 69 131 42 33 113 114 78 77 109 138 155 187 87 119 122 110 58 103 77 84 90 115 163 
111 116 81 82 63 98 46 120 68 107 136 117 131 124 84 98 69 102 91 79 163 96 53 175 104 1

E. Los enfoques B y D demuestran una eficiencia mayor, con resultados entre los 0.15 y 0.20 ms, muy a diferencia del enfoque C, que ronda los 0.80 ms.
Esto es debido a que los enfoques B y D procesan elementos contiguos, mientras que el enfoque C, al funcionar de forma opuesta, hace muchas llamadas a memoria; lo que reduce severamente su eficiencia.

## Ejercicio 2: Multiplicación matriz-vector

In [31]:
%%writefile ej32.cu

#include <iostream>

__global__
void matvec_mult(float* C, float* A, float* B, int n){
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i >= n) return;

  C[i] = 0;
  for(int j = 0; j < n; ++j)
    C[i] += A[j] * B[i * n + j];
}

int main(){
  int n = 128;

  float *host_A, *host_B, *host_C;
  float *dev_A, *dev_B, *dev_C;

  host_A = (float*)malloc(n * sizeof(float));
  host_B = (float*)malloc(n * n * sizeof(float));
  host_C = (float*)malloc(n * sizeof(float));

  for(int i = 0; i < n * n; i++)
    host_B[i] = rand() % 100;

  for(int i = 0; i < n; i++)
    host_A[i] = rand() % 100;

  cudaMalloc(&dev_A, n * sizeof(float));
  cudaMalloc(&dev_B, n * n * sizeof(float));
  cudaMalloc(&dev_C, n * sizeof(float));

  cudaMemcpy(dev_A, host_A, n * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(dev_B, host_B, n * n * sizeof(float), cudaMemcpyHostToDevice);

  dim3 block(32, 16);
  dim3 grid((n + block.x - 1)/block.x,
            (n + block.y - 1)/block.y);

  //--------launch the kernel--------
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  cudaEventRecord(start);

  matvec_mult<<<grid, block>>>(dev_C, dev_A, dev_B, n);

  cudaDeviceSynchronize();

  cudaEventRecord(stop);
  cudaEventSynchronize(stop);

  float ms = 0.0f;
  cudaEventElapsedTime(&ms, start, stop);

  std::cout << "Tiempo GPU: " << ms << " ms\n";

  cudaEventDestroy(start);
  cudaEventDestroy(stop);
  //---------------------------------

  // transferring the output data to host
  cudaMemcpy(host_C, dev_C, n * sizeof(float), cudaMemcpyDeviceToHost);

  // test
  for(int i = 0; i < n; ++i)
    std::cout << host_C[i] << " ";
  std::cout << std::endl;

  // freeing the device memory for the input and output data
  cudaFree(dev_A);
  cudaFree(dev_B);
  cudaFree(dev_C);

  free(host_A);
  free(host_B);
  free(host_C);

  return 0;
}

Overwriting ej32.cu


In [33]:
%%shell

nvcc -arch=compute_75 -code=sm_75 ej32.cu -o ej32
./ej32

Tiempo GPU: 0.331584 ms
339256 293831 339858 314693 338538 314165 309950 325731 339905 320913 299090 347134 337896 327265 308536 315437 323539 336560 288752 323655 335094 284233 321310 350415 305218 327438 339252 339054 321755 299451 319134 323836 301340 331517 345839 306805 330073 323514 324789 330268 299614 328449 298291 352378 310297 358291 338345 343979 319466 306721 313053 324258 335405 329397 302113 292047 329145 327278 308157 306420 306228 323086 332734 340771 285350 306392 328939 320279 346425 293508 327631 319770 310430 312746 303619 278889 338440 339934 311616 291768 337200 297482 352635 311755 321686 311662 328661 302542 320070 297913 325870 286597 269489 334599 323234 340668 292385 345585 343543 315671 319799 317143 304494 346546 321890 329356 282071 307404 310072 343260 301628 297791 317684 291313 346762 312728 352614 321814 299102 330652 319777 314180 334834 336283 310360 321552 313328 306435 


## Ejercicio 3:
Si el SM de un device CUDA puede tomar hasta 1536 threads y 4 blocks de threads, ¿cuál de las siguientes configuraciones de bloques resultaría en el mayor número de threads en el SM?

1. 128 threads per block
2. 256 threads per block
3. 512 threads per block
4. 1024 threads per block

*Respuesta*:

Para la opción 1:

$\frac{\text{1536 threads}}{\text{128 threads por block}} = = \text{12 blocks de threads (máximo 4)}$

$4 \times 128 = 512$

- Habrán 512 threads en el SM

Para la opción 2:

$\frac{\text{1536 threads}}{\text{256 threads por block}} = \text{6 blocks de threads (máximo 4)}$

$4 \times 256 = 1024$

- Habrán 1024 threads en el SM

Para la opción 3:

$\frac{\text{1536 threads}}{\text{512 threads por block}} = \text{3 blocks de threads}$

$3 \times 512 = 1536$

- Habrán 1536 threads en el SM

Para la opción 4:

$\frac{\text{1536 threads}}{\text{1024 threads por block}} = = \text{1 block de threads}$

$1 \times 1024 = 1024$

- Habrán 1024 threads en el SM

Por lo tanto, la configuración con mayor número de threads será la **opción 3**.

## Ejercicio 4:
Para una adición de vectores, asume que la longitud del vector es 2000, cada thread calcula un elemento de salida, y el tamaño del block de threads es de 512 threads. ¿Cuántos threads habrá en el grid?

1. 2000
2. 2024
3. 2048
4. 2096

*Respuesta*:

$\lceil\frac{\text{2000 elementos}}{\text{512 threads por block}}\rceil = \text{4 blocks de threads}$

$4 \times 512 = 2048$

Por lo tanto, habrán 2048 threads (**opción 3**).

## Ejercicio 5:
Con relación a la pregunta anterior, ¿cuántos warps esperarías que tengan divergencia debido al chequeo de límites en la longitud del vector?

*Respuesta*:

$\text{2048 threads} - \lceil\frac{2000}{512}\rceil \text{ bloques de threads} \times 512 = \text{48 bloques}$

$\lceil\frac{48}{32}\rceil = 2$ (32 warps por SM)

Serán 2 warps los que diverjan debido al chequeo de límites.

## Ejercicio 6:
Necesitas implementar un kernel que opere sobre una imagen de 400 x 900 pixeles. Te gustaría asignar un thread a cada pixel. Además, te gustaría que tus blocks de threads sean cuadrados y que usen el máximo número de threads posibles por block en el device (con 3.0 de compute capability). ¿Cómo seleccionarías las dimensiones del grid y del block de tu kernel?

*Respuesta*:

Especificaciones para compute capability 3.0:

Máximo número de threads por block: 1024.
Máximo número de threads por SM: 2048.
Máximo número de blocks por SM: 16.

Para aprovechar el máximo número de threads por block, el producto de las dimensiones debe ser igual a 1024. Si, además, queremos que los blocks sean cuadrados:

$\sqrt{1024} = 32$

Por lo que la dimensión de nuestro block será de $32 \times 32$.

Del mismo modo, las dimensiones del grid serán:
$\lceil\frac{400}{32}\rceil = 13$, $\lceil\frac{900}{32}\rceil = 29$

## Ejercicio 7:
Con relación a la pregunta anterior, ¿Cuántos threads dormidos esperas que hayan?

*Respuesta*:

$ (13 \times 32 - 400) \times (29 \times 32 - 900) = 16 \times 28 = \text{448 threads dormidos.}$